<table style="margin: 0; text-align: left; background-color: #050A0E; border: 1px solid lightblue;">
    <tr>
        <td style="direction: rtl; text-align: right; color: #FFF8E7; ">
            <h2 style="color:#FFF8E7;">JobTonic</h2>
            <span style="color:#FFF8E7; font-size: 12px">
                כי חיפוש עבודה ראוי להרגיש כמו התחלה חדשה, לא כמו מסע מייגע.
                במקום להיסחף בגלים של מידע, 
                תנו לנו להיות הרוח שמכוונת את המפרש.
                הכלי
                 מזקק עבורכם את מהות כל משרה — בבהירות, במהירות, ובדיוק שמחזיר לכם שליטה.
                זה לא רק לחפש עבודה. זה לדייק מטרה. זה להתקדם עם ראש צלול, לב פתוח, וביטחון אמיתי.
                בכל שלב בדרך – אנחנו האנרגיה שמזיזה אתכם קדימה.
            </span>
        </td>
<td style="width: 290px; height: 200px; vertical-align: middle;">
            <img src="JobTonic.png" style="width: 100%; height: 100%; display: block;" />
        </td>
    </tr>
</table>

In [83]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import time
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
import ollama
from PIL import Image
import pytesseract

In [84]:
# Constants
MODEL = "llama3.2"

# Load environment variables
load_dotenv(override=True)

# Initialize Google and Anthropic clients
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
MODEL_Claude = "claude-3-7-sonnet-latest"
claude = anthropic.Anthropic()

google_api_key = os.getenv('GOOGLE_API_KEY')
MODEL_Google = "gemini-2.0-flash"
google.generativeai.configure()

In [85]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with Selenium for dynamic websites.
    """

    def __init__(self, url):
        self.url = url

        # Set up Selenium WebDriver with Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run in headless mode
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # Load the webpage
        driver.get(url)
        time.sleep(5)  # Wait for the page to load completely

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()  # Close the browser

        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [86]:
# generic API call to 3 models
def callModel(company, system_prompt, user_prompt):
    print(f"Calling {company}")
    if company == "ollama":
        response = ollama.chat(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
        )
        return response['message']['content']
    elif company == "anthropic":
        response = claude.messages.create(
            model=MODEL_Claude,
            max_tokens=500,
            temperature=0.4,
            system=system_prompt,
            messages=[
                {"role": "user", "content": user_prompt},
            ],
        )
        return response.content[0].text
    elif company == "google":
        gemini = google.generativeai.GenerativeModel(
            model_name=MODEL_Google,
            system_instruction=system_prompt,
            )
        response = gemini.generate_content(user_prompt)
        return response.text


In [87]:
# The system prompt for job postings
job_posting_system_prompt = (
    "You are an assistant specialized in summarizing job postings.\n"
    "Your task is to extract and clearly summarize three things from the input job description:\n"
    "1. How the company describes itself.\n"
    "2. What the day-to-day responsibilities of the role are.\n"
    "3. What the qualifications or requirements are to apply.\n\n"
    "Write the summary in plain, professional English. Keep each section concise (1–2 sentences).\n"
    "Do not copy text verbatim unless necessary. Structure your response under the following headers:\n"
    "- About the Company\n"
    "- Role Responsibilities\n"
    "- Qualifications and Requirements\n"
    "in addition to the following fields:\n"
    "- Company Name\n"
    "- Job URL\n"
    "- Company Location\n"
    "- Company Size\n"
)
job_posting_system_prompt += "\nYou should respond in JSON as in this example:"
job_posting_system_prompt += """
{
    "company_name": "Tech Innovations Inc.",
    "job_url": "https://full.url/goes/here/software-engineer",
    "company_location": "San Francisco, CA",
    "company_size": "500+ employees",
    "company_description": "The company is a leading provider of innovative technology solutions.",
    "role_responsibilities": "The role involves developing software applications and collaborating with cross-functional teams.",
    "qualifications_requirements": "Candidates should have a degree in Computer Science and experience with Python."
}
"""
job_posting_system_prompt += "\nIn the case that there are no relevant links, respond with an empty JSON object: {}"

job_posting_system_prompt += "\n here is a real example of a job posting:\n"
job_posting_system_prompt += """
{
  "company_name": "Earnix",
  "job_url": "https://earnix.com/career/0d.f45/automation-engineer/",
  "company_location": "Ramat Gan, Israel",
  "company_size": "201–500 employees",
  "company_description": "Earnix is a premier provider of cloud-based intelligent decisioning solutions for pricing, rating, underwriting, and product personalization in the insurance and banking sectors, serving clients across over 35 countries.",
  "role_responsibilities": "Develop and enhance server-side automation tests and infrastructure, analyze test results, participate in code reviews, and collaborate with cross-functional teams to ensure high-quality product delivery.",
  "qualifications_requirements": "Minimum 3 years of experience in server-side automation development using Python/Pytest, proficiency with AWS, Docker, Jenkins, Git, and Linux environments, a bachelor's degree in Computer Science or related field, with a background in mathematics or statistics considered an advantage."
}
"""

job_posting_system_prompt += "\n here is another real example of a job posting:\n"
job_posting_system_prompt += """
{
  "company_name": "Earnix",
  "job_url": "https://earnix.com/career/0d.f45/automation-engineer/",
  "company_location": "Givatayim, Israel",
  "company_size": "201-500 employees",
  "company_description": "Earnix is a global provider of AI-driven rating, pricing, and product personalization solutions for insurance and banking, helping financial institutions transform how they make decisions with real-time, dynamic, and integrated analytics.",
  "role_responsibilities": "The Automation Engineer will develop and maintain test automation frameworks, create and execute automated test scripts, collaborate with development teams on CI/CD pipelines, and continuously improve testing methodologies to ensure high-quality software releases.",
  "qualifications_requirements": "Candidates need 3+ years of experience in test automation, proficiency in Python, knowledge of API testing frameworks like REST-assured or Postman, familiarity with CI/CD tools, and strong analytical and problem-solving skills."
}
"""

In [96]:
# This function is used to create a user prompt to find the job postings
def get_job_postings_user_prompt(text):
    user_prompt = "Please analyze the following job description:\n\n"
    user_prompt += text
    return user_prompt

In [97]:
is_job_relevant_system_prompt = """ You will receive:
1. A job description summary (including role, requirements, and expectations).
2. A resume in free-text/plain-text format.

Your task is to evaluate whether the job appears relevant to the candidate based on the skills, experiences, and qualifications in the resume.

Instructions:
- Compare the core requirements and desired qualifications from the job description to the candidate's experience,
 skills, education, and achievements in the resume.
- Focus on key match indicators such as:
  - Required technologies, tools, or methodologies
  - Years of experience
  - Industry relevance
  - Certifications or degrees
  - Role-specific accomplishments

Respond with the following structure:
```json
{
  "relevant": true or false,
  "reasoning": "A concise explanation of why the job is or isn't relevant to the candidate."
}"""


In [98]:
def get_job_relevant_user_prompt(resume, job_summary):
    user_prompt = "Here is the job description summary and the candidate's resume. " \
    "Please assess whether the job is relevant to the candidate."
    user_prompt += """The resume: \n\n"""
    user_prompt += resume
    user_prompt += """\n\n
    The job description: \n\n"""
    user_prompt += job_summary
    # print(user_prompt)
    return user_prompt

In [99]:
from docx import Document

def read_docx_text(filepath):
    try:
        doc = Document(filepath)
        text = '\n'.join([para.text for para in doc.paragraphs if para.text.strip() != ''])
        return text
    except Exception as e:
        return f"Error reading DOCX: {e}"


In [100]:
import fitz  # PyMuPDF

def read_pdf_text(filepath):
    try:
        text = ""
        with fitz.open(filepath) as doc:
            for page in doc:
                text += page.get_text()
        return text
    except Exception as e:
        return f"Error reading PDF: {e}"


In [102]:
def read_text_file(file_name):
    try:
        with open(file_name, "r", encoding="utf-8") as f:
            text = f.read()
        return text
    except Exception as e:
        return f"Error reading text file: {e}"

In [106]:
# This function get a url of job posting and answer the user if it can apply to it
def job_agent(resume_file, txt):
    # Read the resume file
    resume = read_pdf_text(resume_file)
    # Call text model to get the job posting summary
    job_summary = callModel("anthropic", job_posting_system_prompt, get_job_postings_user_prompt(txt))
    # Call the model to check if the job is relevant
    is_relevant = callModel("anthropic", is_job_relevant_system_prompt, get_job_relevant_user_prompt(resume, job_summary))
    answer = job_summary + "\n\n" + is_relevant
    return answer

In [104]:
resume_file = "SapirShamay.pdf"
print(read_pdf_text(resume_file))

EDUCATION
BEN-GURION UNIVERSITY 
BSc in Software and Information System
Engineering (Avg - 83)
2018 - 2022
2025 - Current
LLM ENGINEERING: MASTER AI, LARGE
LANGUAGE MODELS & AGENTS
CONTACT
(972) 50 6910190
shamay.sapir@gmail.com
linkedin.com/in/sapir-shamay
WORK EXPERIENCE
2021-2025
Simplex (by Nuvei)
Senior R&D analyst
2019-2020
Wix
Software Engineer
2013-2015
MAMRAM, IDF
Software Developer, Team Leader - Captain
Tech lead and backend engineer in the company’s payments infrastructure, responsible for
mission-critical components powering hundreds of thousands of transactions a month.
Designed and implemented high-impact, cross-team features in Python, deployed to internal
payment gateways at scale, leading to a 0.5% uplift in conversion rate.
Led the full technical lifecycle: authored design docs, set Git flow standards, enforced PEP8 with
linters, increased test coverage by 5%, and streamlined the code review pipeline.
Working across multiple environments including production, simulat

In [105]:
# Example usage
job_search_url = "https://www.factify.com/careers"
job_file = "jobDescription.txt"
resume_file = "SapirShamay.pdf"

In [ ]:
# if website
job_search_url = "https://www.factify.com/careers"
website = Website(job_search_url)
answer = job_agent(resume_file, website.text)
print(answer)

In [107]:
# if file
job_file = "jobDescription.txt"
answer = job_agent(resume_file, read_text_file(job_file))
print(answer)


Calling anthropic
Calling anthropic
Calling anthropic
{
  "company_name": "Factify",
  "job_url": "",
  "company_location": "Israel",
  "company_size": "",
  "company_description": "Factify is an early-stage Silicon Valley company developing an enterprise document platform that transforms documents through their Smarter Document format, disrupting a $500 billion industry with revolutionary document experience, applications, and enterprise AI.",
  "role_responsibilities": "The Backend Engineer will design and implement production-grade APIs, build scalable microservices using Python and Go, develop AI infrastructure components, create robust data persistence layers, and implement observability and monitoring solutions across distributed systems.",
  "qualifications_requirements": "Candidates need 3+ years of experience building distributed backend systems, strong proficiency in Python and Go, experience with gRPC/REST APIs, hands-on experience with cloud platforms and containerization t

In [1]:
system_prompt ="You are an expert resume writer specializing in tailoring resumes to specific" \
" job descriptions. Given the following resume document provided  and the following job description" \
" in this url: https://earnix.com/career/ae.559/junior-software-developer-genai-systems/, " \
"rewrite the resume text to:* Prioritize skills and experiences most relevant to the job description." \
"* Incorporate keywords and phrases from the job description to optimize for Applicant Tracking Systems" \
" (ATS).* Use strong action verbs and professional language.* Maintain the original structure" \
" and information hierarchy of the resume.* Focus on rephrasing the content, " \
"do not add new information not present in the original resume.* " \
"Please rewrite this document in plain text format only, keeping the original " \
"section titles (like Profile, Employment History, etc.). Use blank lines to " \
"separate roles or sections. Do not include explanations or markdown. This will be converted directly " \
"into a .docx file later."

In [ ]:
response = job_postings("google", job_posting_system_prompt, user_prompt)
print(response)

```json
{
  "company_name": "Earnix",
  "job_url": "https://earnix.com/career/",
  "company_location": "Ramat Gan, IL",
  "company_size": "201-500 employees",
  "company_description": "Earnix is a premier provider of mission-critical, cloud-based intelligent decisioning across pricing, rating, underwriting, and product personalization solutions. They offer fully-integrated solutions designed to transform how global insurers and banks are run.",
  "role_responsibilities": "The Fullstack BI Developer will design and develop full-stack BI solutions from database to dashboards, build and maintain ETL pipelines and data models, and work with cross-functional teams to deliver impactful analytics.",
  "qualifications_requirements": "The role requires 3-5 years of experience in both frontend and backend BI development, a bachelor's degree in a relevant field, a strong understanding of data warehousing concepts and ETL processes, and experience with Snowflake, Tableau, and SQL. Experience with 